###  Naïve Bayes (NB) classification algorithm - Assignment 3 (AMOD 5310H)


In [26]:
# Step 1: Import necessary libraries
import pandas as pd                         # For reading CSV files
import sklearn
import numpy
from sklearn.model_selection import train_test_split  # For splitting the dataset
from sklearn.metrics import accuracy_score            # For evaluating performance
from collections import defaultdict                   # For managing nested dictionaries



In [27]:

# Step 2: Load your dataset
# for understanding of the steps required[1],[2].
df = pd.read_csv("sample_dataset.csv")

# Step 3: Split the dataset

# reference of how to use iloc [3],[4],[5]

X = df.iloc[:, :-1] # all columns except the last[selects all rows and all columns except the last one (i.e., the 4 features).]
y = df.iloc[:, -1] # the last column[selects all rows of the last column only, which is the target (Occupied).]

# split into training and test sets
# reference of where I learned to do the following[6],[7].
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size= 0.2, random_state= 42 
) 
# random_state=42: makes the split reproducible (you get the same split every time you run it)


In [ ]:
# Step 4: Calculate Prior Probabilities
"""
Prior probabilities
P = counts of class y/ total number of samples in class y

"""


# get the frequency of each counts: Yes/No(classes).
# reference for the following [8].
counts = y_train.value_counts()
print(f"\nthe following are the freqency of each class:")
print(f"\n The number of Yes is: {counts['Yes']}")
print(f"\n The number of NO is: {counts['No']}")


#  total number of training samples in class y
total_counts = len(y_train) 
print("**************************")
print(f"\nthe total number of classes: {total_counts}")

# Divide each count by the total number of training samples.
#p_yes = counts['Yes'] / total_counts
#p_no = counts['No'] / total_counts

# prior probability
#priors = {
#    "Yes": p_yes, 
#    "No": p_no
#}

# Automatically calculate priors for any class in the dataset
priors = {}

for cls in counts.index:
    priors[cls] = counts[cls] / total_counts


print("\nPrior Probabilities:")
for cls, prob in priors.items():
    print(f"Class: {cls}, P({cls}) = {prob:.4f}")

"""
print("**************************")

print(
    "\nPrior probabilities becimes:",
    f"\nThe probability of Yes is: {priors['Yes']:.4f}",
    f"\nThe probability of No is: {priors['No']:.4f}"
)

print("**************************")

"""


the following are the freqency of each class:

 The number of Yes is: 7

 The number of NO is: 4
**************************

the total number of classes: 11

✅ Prior Probabilities:
Class: Yes, P(Yes) = 0.6364
Class: No, P(No) = 0.3636


'\nprint("**************************")\n\nprint(\n    "\nPrior probabilities becimes:",\n    f"\nThe probability of Yes is: {priors[\'Yes\']:.4f}",\n    f"\nThe probability of No is: {priors[\'No\']:.4f}"\n)\n\nprint("**************************")\n\n'

In [ ]:
# Automatically calculate priors for any class in the dataset
priors = {}

for cls in counts.index:
    priors[cls] = counts[cls] / total_counts


In [ ]:
# Step 5: Calculate Conditional probabilities

# reference for understanding how Conditional probabilities works [9].

"""Conditional probability
P = count of( X=x and Y=y happening) / count of class y(it can be counts of Yes or Counts of No)
"""

conditional_probs = {} 
# reference for the following [10].
features = X_train.columns
classes = y_train.unique()


# loop through each feature

for feature in features:

    conditional_probs[feature] = {} # initialize dictionary for storing, for this feature

    unique_vals = X_train[feature].unique() # get all possible values for this feature

    # loop through each value of the current feature

    for val in unique_vals:
        conditional_probs[feature][val] = {} # initialize dictionary to hold class probabilities
        for cls in classes:
            # Get all rows in X_train where class == cls
            # also for the following a for loop would have been much more intuitive, but the code would get to long.
            rows_with_class = X_train[y_train == cls] # this will give you the rows where you have either Yes/No, for a given feature
            #print(f"\nthe following are all rows in X_train, where the label is Yes (or No): {rows_with_class}")
            # which allow to know how often each feature value (e.g., "Sunny") appears with a specific class  (e.g., "Yes")
            #print("****************************************************************")


            # Count how many times feature value == val in those rows
            count_feature_val = (rows_with_class[feature] == val).sum()  
            #print(f"\nthe following are how many times feature value == val in those rows: {count_feature_val}")
            # It basically counts how many times the current feature has a specific value (like "Sunny") 
            # for a specific class (cls = "Yes" or "No").
            #print("****************************************************************")


            # Count total number of row8-Puzzle A Searchs where class == cls
            count_class = (y_train == cls).sum()
            #print(f"\nthe following are the rows where class == cls: {count_class}")
            # This counts how many total times the class label cls (e.g., "Yes" or "No") appears in the training set
            # for a specific value.
            #print("****************************************************************")


            
            prob = count_feature_val / count_class
            conditional_probs[feature][val][cls] = prob
print(conditional_probs['Outlook']) # to get an idea of how it looks like
print("****************************************************************")


{'Overcast': {'Yes': 0.42857142857142855, 'No': 0.0}, 'Rain': {'Yes': 0.2857142857142857, 'No': 0.5}, 'Sunny': {'Yes': 0.2857142857142857, 'No': 0.5}}
****************************************************************


In [ ]:
# Step 6: Put everything together 

def predict_instance(instance, priors, conditional_probs):
    """
    # here an instance is one full example from the dataset.
    # it is all the feature values that describe one thing you're trying to classify.
    # Outlook = Sunny, Temperature = Cool, Humidity = High, Wind = Strong
    # it is colum and value in that colum for that row.
    
    """
    
    posteriors={} # to store probability for each class
    # loop through each possible class. the goal is to check the prior probability( how likely I am to find it)
    for cls in priors:
        print(f"\nEvaluating class: {cls}")
        prob = priors[cls] # this use the priors defined earlier in step 4
        #print(f"Prior P({cls}) = {prob:.4f}")

    # multiply with the conditional probabilities for each feature.
    # to do that. first for each feature/column for a given row, grab the feature and value.
        for feature in instance.index:
            value = instance[feature] # the value of that feature in the instance
            # which is slightly confusing because above I never created an instance list/dictionary
            # but several hours of explantion from Gemini, conviced me that the code
            # will know what it is doing, even if my brain is lagging behind.
            
            # this is the same as the conditional probability created above.
            cond_prob = conditional_probs[feature][value][cls]
            print(f"Likelihood P({feature}={value} | {cls}) = {cond_prob:.4f}")

            # multiply the current probability(priors) to the conditional probability(cond_prob)
            new_prob = prob * cond_prob
            prob = new_prob



        print(f"  Probability for each class {cls}: {prob:.4}")
        posteriors[cls]= prob

    # choose the class with the highest posterior
    # reference for the following [11].
    predicted_class = max(posteriors, key = posteriors.get)
    print(f"\nPredicted class: {predicted_class} with probability {posteriors[predicted_class]:.6f}")
    return predicted_class


In [ ]:
# Step 7: Predict on the test set

y_pred_prob = []

# reference for the following [12].
for index, row in X_test.iterrows():
    prediction = predict_instance(row, priors, conditional_probs)
    y_pred_prob.append(prediction)



Evaluating class: Yes
Likelihood P(Outlook=Rain | Yes) = 0.2857
Likelihood P(Temperature=Mild | Yes) = 0.2857
Likelihood P(Humidity=Normal | Yes) = 0.7143
Likelihood P(Wind=Weak | Yes) = 0.5714
  Probability for each class Yes: 0.0212

Evaluating class: No
Likelihood P(Outlook=Rain | No) = 0.5000
Likelihood P(Temperature=Mild | No) = 0.5000
Likelihood P(Humidity=Normal | No) = 0.2500
Likelihood P(Wind=Weak | No) = 0.2500
  Probability for each class No: 0.005682

Predicted class: Yes with probability 0.021203

Evaluating class: Yes
Likelihood P(Outlook=Overcast | Yes) = 0.4286
Likelihood P(Temperature=Mild | Yes) = 0.2857
Likelihood P(Humidity=High | Yes) = 0.2857
Likelihood P(Wind=Strong | Yes) = 0.4286
  Probability for each class Yes: 0.009541

Evaluating class: No
Likelihood P(Outlook=Overcast | No) = 0.0000
Likelihood P(Temperature=Mild | No) = 0.5000
Likelihood P(Humidity=High | No) = 0.7500
Likelihood P(Wind=Strong | No) = 0.7500
  Probability for each class No: 0.0

Predicted 

In [ ]:
# Step 8: Calculate Accuracy
# reference for the following[13]. 
accuracy = accuracy_score(y_test, y_pred_prob)
print(f"\n Model accuracy: {accuracy:.2f}")


 Model accuracy: 1.00
